In [3]:
import cv2
import pytesseract
from pdf2image import convert_from_path
from pdf2jpg import pdf2jpg
from pytesseract import Output
import pandas as pd
import numpy as np
import os
import requests
from json import JSONDecodeError
import json
import time
from requests.exceptions import ConnectionError




# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

#skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

#template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)

# def myFunc(df):
#   return df.iloc[0]['row_column']

pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

PATH = r'(location of pdf file here)'

os.chdir(PATH)#filepath

os.chdir(PATH)
for filename in os.listdir(os.getcwd()):
        
        if not os.path.exists('cache'): 
            os.mkdir('cache')
        print(os.getcwd(), filename)
        images = convert_from_path(os.getcwd()+"\\"+filename,poppler_path=r'C:\Program Files\poppler-0.67.0\bin')
        # images = convert_from_path('D:\Downloads New\Edddd.pdf',poppler_path=r'D:\Program Files\poppler-0.67.0\bin')
        imagecount = 0
        raw=[]
        final = []
        finalpd =pd.DataFrame()
        for img in images:
                imagecount += 1
                if(imagecount != 1 and imagecount != 2 and imagecount != len(images)):
#                 if imagecount == 23  :
                        # cv2.imshow("one pic", np.uint8(img))
                        # cv2.waitKey(0)
                        

                        print(imagecount)
                        img = np.uint8(img)


                        # img = cv2.imread(r'D:/pythonProjects/1.jpg')
                        himage,wimage = img.shape[:-1]
                        # img = cv2.imread(r'G:/project/OCR/FDA.png')
                        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                        cong = r' --oem 3 --psm 12 -l eng'

                        h,w = img.shape[:-1]
                        # c = img[115:200, 67:500]
                        # print(pytesseract.image_to_data(c,config=cong))


                        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]



 


    



                       
                       
                       
                       
                        # thresh = thresholding(gray)
                        #Remove horizondal lines

                        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40,1))
                        remove_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
                        cnts = cv2.findContours(remove_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
                        for c in cnts:
                                cv2.drawContours(img, [c], -1, (255,255,255), 5)

                # Remove vertical lines
                        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,10))
                        remove_vertical = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel, iterations=2)
                        cnts = cv2.findContours(remove_vertical, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
                        for c in cnts:
                                cv2.drawContours(img, [c], -1, (255,255,255), 5)


                        
#                         cv2.imshow("one pic", img)

#                         cv2.waitKey(0)


                        bl = pytesseract.image_to_data(img,config=cong, output_type='data.frame')
#                         print(bl)
#                         c = img[115:200, 67:500]
#                         print(pytesseract.image_to_data(c,config=cong))
#                         g = pytesseract.image_to_data(c,config=cong, output_type='data.frame')
#                         print(g)
#                         bl.to_csv("bl.csv",index=False)
                        raw.append(bl)
                        aa = bl[(bl.word_num == 1)& ((bl.width.between(103,140)) | (bl.width.between(170,185))) & (bl.left >= 180) & (bl.height < 22) & (bl.conf != -1)  ]
#                         aa.to_csv('aa.csv')
                        final.append(aa)
#                         print(final)
        
        final = pd.concat(final)
        raw = pd.concat(raw)
        final['char'] = final['text'].str[:3]
        final['digit'] = final['text'].str[3:]

        final['char'] = final['char'].str.upper()
        final['char'] = final['char'].str.replace('1','I')
        final['char'] = final['char'].str.replace('|','I')
        final['char'] = final['char'].str.replace('2','Z')
        final['char'] = final['char'].str.replace("\\","I")
        final['char'] = final['char'].str.replace('$','S')
        final['char'] = final['char'].str.replace('0','O')

        final['digit'] = final['digit'].str.replace('O','0')
        final['digit'] = final['digit'].str.replace('o','0')

        final['EpicNo'] = final['char'] + final['digit']
        final['EpicNo'] = final['EpicNo'].str.replace(',','')
        final['EpicNo'] = final['EpicNo'].str.replace('.','')
        final['EpicNo'] = final['EpicNo'].str.replace("'",'')
        final.to_excel(".//cache//"+"Id_card"+filename[:filename.index(".")]+".xlsx",index=False,engine='openpyxl')      
        raw.to_excel(".//cache//"+"Raw_"+filename[:filename.index(".")]+".xlsx",index=False,engine='openpyxl')      

        
        voterlist = []
        error = []
        for i in final['EpicNo']:
            if (len(i)==11 and i[3] == '0' and i[4] == '0'):
                i = i[0:3]+i[4:]
                try:

                    print(i)
                    epic = requests.get(f'https://www.nvsp.in/Master/GetFamilyDetail/?FamilyEpic={i}')
                    data = epic.json()
                    adict = json.loads(data)
                    check = pd.DataFrame([adict['response']['docs']])
                    if (check.shape[1] == 2):        
                        df = pd.DataFrame([adict['response']['docs'][1]])
                        voterlist.append(df)
                        df = pd.DataFrame([adict['response']['docs'][0]])
                        voterlist.append(df)
                    else:
                        df = pd.DataFrame([adict['response']['docs'][0]])
                        voterlist.append(df)
                except (JSONDecodeError,IndexError):
                    print('error idcard:',i)
                    error.append(i)
                    pass
            else:
                while True:
                    try:

                        print(i)
                        time.sleep(0.03)
                        epic = requests.get(f'https://www.nvsp.in/Master/GetFamilyDetail/?FamilyEpic={i}')
                        data = epic.json()
                        adict = json.loads(data)
                        check = pd.DataFrame([adict['response']['docs']])
                        if (check.shape[1] == 2):        
                            df = pd.DataFrame([adict['response']['docs'][1]])
                            voterlist.append(df)
                            df = pd.DataFrame([adict['response']['docs'][0]])
                            voterlist.append(df)
                        else:
                            df = pd.DataFrame([adict['response']['docs'][0]])
                            voterlist.append(df)
                    except (JSONDecodeError,IndexError):
                        print('error idcard:',i)
                        error.append(i)
                        pass
                    except ConnectionError:
                        continue
                    break
                
        voterlist = pd.concat(voterlist)
        error = pd.DataFrame(error)
        voterlist.to_excel(".//"+filename[:filename.index(".")]+'.xlsx',index=False,engine='openpyxl')
        error.to_excel(".//cache//"+"Error "+filename[:filename.index(".")]+'.xlsx',engine='openpyxl')
        lst = voterlist['slno_inpart'].astype(str).astype(int)
        def find_missing(lst):
            start = lst.iloc[0]
            end = lst.iloc[-1]
            return sorted(set(range(start, end + 1)).difference(lst))

        aa = find_missing(lst)
        aa = pd.DataFrame(aa,columns=['Epic_No'])
        aa.to_excel(".//cache//"+"Missing_Nos"+filename[:filename.index(".")]+'.xlsx',engine='openpyxl')


        


